In [1]:
if 'snakemake' in locals():
   int_file = snakemake.input[0]
   ref_proteome_file = snakemake.input[1]
   output_file = snakemake.output[0]
else:
   int_file = '../../../data/decryptm/10_Kinase_Inhibitors/Phosphoproteome/curves_2KI.txt'
   ref_proteome_file = '../../../data/decryptm/uniprot_proteome_up000005640_03112020.fasta'
   output_file = '../../../results/decryptm/protein_mapping/mapped_protein_2KI.csv'

import pandas as pd
import numpy as np
import re
from Bio import SeqIO

In [2]:
def parse_phosphosite_info(prob_peptide, probability_trheshold=0.75):
    ast_peptide = re.sub('\(.*?\)', '*', prob_peptide)
    seq_peptide = re.sub('\(.*?\)', '', prob_peptide)
    # locate the position before every asterisk
    phospho_positions = [m.start() for m in re.finditer('\*', ast_peptide)]
    phospho_positions = [i - n for n, i in enumerate(phospho_positions)]
    # parse the aminoacid in interesting positions
    phospho_aas = [seq_peptide[i - 1] for i in phospho_positions]
    # extract all phospho probabilities
    phospho_probs = [float(i[1:-1]) for i in re.findall('\(.*?\)', prob_peptide)]
    # filter phospho positions, aas and probs by probability threshold
    phospho_positions = [i for i, j in zip(phospho_positions, phospho_probs) if j >= probability_trheshold]
    phospho_aas = [i for i, j in zip(phospho_aas, phospho_probs) if j >= probability_trheshold]
    phospho_probs = [i for i in phospho_probs if i >= probability_trheshold]
    # if there are no phospho sites, return None
    if len(phospho_positions) == 0:
        return None, None, None
    else:
        return phospho_positions, phospho_aas, phospho_probs

In [5]:
#read fasta file as a dictionary
ref_proteome = SeqIO.to_dict(SeqIO.parse(ref_proteome_file, "fasta"))

# replace IDs by the second element after splitting by '|'
ref_proteome = {k.split('|')[1]: v for k, v in ref_proteome.items()}

# read tsv file
df = pd.read_csv(int_file, sep='\t')
protein_id_column = 'Leading proteins'
phospho_count_column = 'Phospho (STY)'
phospho_prob_column = 'All Phospho (STY) Probabilities'
seq_column = 'Sequence'

# remove all rows on which phospho count is 0
df = df[df[phospho_count_column] != 0].copy()
filt_df = df.iloc[:, :].copy()

# parse phosphosite info
psite_location = []
peptide_start = []

In [7]:
# iterate over the rows of the df
for index, row in df.iterrows():
    phospho_count = int(row[phospho_count_column])
    phospho_prob_pep = row[phospho_prob_column].split(';')[0]
    lead_prot = row[protein_id_column].split(';')[0]

In [8]:
pos, aa, prob = parse_phosphosite_info(phospho_prob_pep, 0.75)
# keep only the phospho_count positions and aminoacids with the highest probability
int_psite_index = np.argsort(prob)[-phospho_count:]
int_pos = [pos[i] for i in int_psite_index]
int_aa = [aa[i] for i in int_psite_index]

In [9]:
ref_seq = ref_proteome[lead_prot].seq

In [10]:
start_pos = ref_seq.find(row[seq_column]) + 1
# add the start position to the phosphosite position
int_pos = [i + start_pos -1 for i in int_pos]
# concatenate aa and positions without separator
out_psite = ';'.join([i + str(j) for i, j in zip(int_aa, int_pos)])

In [27]:
start = [x - 8 for x in int_pos]

In [28]:
for i in start:
    full_seq = ref_seq[i:(i+15)]


In [29]:
full_seq

Seq('YSKYYSDSDDELTVE')

In [ ]:
psite_location.append(out_psite)
peptide_start.append(start_pos)

In [51]:
start_pos = ref_seq.find(row[seq_column]) + 1
# add the start position to the phosphosite position
int_pos = [i + start_pos -1 for i in int_pos]
# concatenate aa and positions without separator
out_psite = ';'.join([i + str(j) for i, j in zip(int_aa, int_pos)])
psite_location.append(out_psite)
peptide_start.append(start_pos)

In [52]:
psite_location

['S484', 'S963']

In [3]:
ref_proteome = SeqIO.to_dict(SeqIO.parse(ref_proteome_file, "fasta"))

# replace IDs by the second element after splitting by '|'
ref_proteome = {k.split('|')[1]: v for k, v in ref_proteome.items()}

In [4]:
# read tsv file
df = pd.read_csv(int_file, sep='\t')
protein_id_column = 'Leading proteins'
phospho_count_column = 'Phospho (STY)'
phospho_prob_column = 'All Phospho (STY) Probabilities'
seq_column = 'Sequence'

# remove all rows on which phospho count is 0
df = df[df[phospho_count_column] != 0].copy()
filt_df = df.iloc[:, :].copy()

# parse phosphosite info
psite_location = []
peptide_start = []
# iterate over the rows of the df

NameError: name 'phospho_prob_pep' is not defined

In [6]:
lead_prot

'Q8NFC6'

In [7]:
pos, aa, prob = parse_phosphosite_info(phospho_prob_pep, 0.75)

In [11]:
prob_peptide = phospho_prob_pep

In [13]:
ast_peptide = re.sub('\(.*?\)', '*', prob_peptide)
seq_peptide = re.sub('\(.*?\)', '', prob_peptide)

In [18]:
phospho_positions = [m.start() for m in re.finditer('\*', ast_peptide)]
phospho_positions = [i - n for n, i in enumerate(phospho_positions)]
phospho_positions

[3, 5]

In [20]:
phospho_aas = [seq_peptide[i - 1] for i in phospho_positions]

In [21]:
phospho_aas

['S', 'S']

In [22]:
phospho_probs = [float(i[1:-1]) for i in re.findall('\(.*?\)', prob_peptide)]

In [23]:
phospho_probs

[0.198, 0.802]

In [25]:
probability_trheshold = 0.75

In [26]:
phospho_positions = [i for i, j in zip(phospho_positions, phospho_probs) if j >= probability_trheshold]

In [27]:
phospho_aas = [i for i, j in zip(phospho_aas, phospho_probs) if j >= probability_trheshold]
phospho_probs = [i for i in phospho_probs if i >= probability_trheshold]

In [34]:
seq_peptide

'YYSDSDDELTVEQR'

In [ ]:
ast_peptide = re.sub('\(.*?\)', '*', prob_peptide)
seq_peptide = re.sub('\(.*?\)', '', prob_peptide)
# locate the position before every asterisk
phospho_positions = [m.start() for m in re.finditer('\*', ast_peptide)]
phospho_positions = [i - n for n, i in enumerate(phospho_positions)]
# parse the aminoacid in interesting positions
phospho_aas = [seq_peptide[i - 1] for i in phospho_positions]
# extract all phospho probabilities
phospho_probs = [float(i[1:-1]) for i in re.findall('\(.*?\)', prob_peptide)]
# filter phospho positions, aas and probs by probability threshold
phospho_positions = [i for i, j in zip(phospho_positions, phospho_probs) if j >= probability_trheshold]
phospho_aas = [i for i, j in zip(phospho_aas, phospho_probs) if j >= probability_trheshold]
phospho_probs = [i for i in phospho_probs if i >= probability_trheshold]
# if there are no phospho sites, return None
if len(phospho_positions) == 0:
    return None, None, None
else:
    return phospho_positions, phospho_aas, phospho_probs

In [9]:
aa

['S']

In [10]:
prob

[0.802]

In [102]:
if 'snakemake' in locals():
   int_file = snakemake.input[0]
   ref_proteome_file = snakemake.input[1]
   output_file = snakemake.output[0]
else:
   int_file = '../../../data/decryptm/10_Kinase_Inhibitors/Phosphoproteome/curves_2KI.txt'
   ref_proteome_file = '../../../data/decryptm/uniprot_proteome_up000005640_03112020.fasta'
   output_file = '../../../results/decryptm/protein_mapping/mapped_protein_2KI.csv'

import pandas as pd
import numpy as np
import re
from Bio import SeqIO
from Bio.Seq import Seq

def parse_phosphosite_info(prob_peptide, probability_trheshold=0.75):
    ast_peptide = re.sub('\(.*?\)', '*', prob_peptide)
    seq_peptide = re.sub('\(.*?\)', '', prob_peptide)
    # locate the position before every asterisk
    phospho_positions = [m.start() for m in re.finditer('\*', ast_peptide)]
    phospho_positions = [i - n for n, i in enumerate(phospho_positions)]
    # parse the aminoacid in interesting positions
    phospho_aas = [seq_peptide[i - 1] for i in phospho_positions]
    # extract all phospho probabilities
    phospho_probs = [float(i[1:-1]) for i in re.findall('\(.*?\)', prob_peptide)]
    # filter phospho positions, aas and probs by probability threshold
    phospho_positions = [i for i, j in zip(phospho_positions, phospho_probs) if j >= probability_trheshold]
    phospho_aas = [i for i, j in zip(phospho_aas, phospho_probs) if j >= probability_trheshold]
    phospho_probs = [i for i in phospho_probs if i >= probability_trheshold]
    # if there are no phospho sites, return None
    if len(phospho_positions) == 0:
        return None, None, None
    else:
        return phospho_positions, phospho_aas, phospho_probs
    
# read fasta file as a dictionary
ref_proteome = SeqIO.to_dict(SeqIO.parse(ref_proteome_file, "fasta"))

# replace IDs by the second element after splitting by '|'
ref_proteome = {k.split('|')[1]: v for k, v in ref_proteome.items()}

# read tsv file
df = pd.read_csv(int_file, sep='\t')
protein_id_column = 'Leading proteins'
phospho_count_column = 'Phospho (STY)'
phospho_prob_column = 'All Phospho (STY) Probabilities'
seq_column = 'Sequence'

# remove all rows on which phospho count is 0
df = df[df[phospho_count_column] != 0].copy()
filt_df = df.iloc[:, :].copy()

# parse phosphosite info
psite_location = []
peptide_start = []
fifteenmer = []
# iterate over the rows of the df
for index, row in df.iterrows():
    phospho_count = int(row[phospho_count_column])
    phospho_prob_pep = row[phospho_prob_column].split(';')[0]
    lead_prot = row[protein_id_column].split(';')[0]
    pos, aa, prob = parse_phosphosite_info(phospho_prob_pep, 0.75)
    # if the output of the parsing is None, then append an empty string to psite_location and psite_start
    if pos is None:
        psite_location.append('')
        peptide_start.append('')
        fifteenmer.append('')
        continue
    # keep only the phospho_count positions and aminoacids with the highest probability
    int_psite_index = np.argsort(prob)[-phospho_count:]
    int_pos = [pos[i] for i in int_psite_index]
    int_aa = [aa[i] for i in int_psite_index]
    # locate start position of the peptide in the protein
    # if the lead protein is not a key in the ref_proteome dictionary, skip the row and append an empty string
    if lead_prot not in ref_proteome:
        psite_location.append('')
        peptide_start.append('')
        fifteenmer.append('')
        continue
    ref_seq = ref_proteome[lead_prot].seq
    start_pos = ref_seq.find(row[seq_column]) + 1
    # add the start position to the phosphosite position
    int_pos = [i + start_pos -1 for i in int_pos]
    # get fifteenmer
    out = []
    for i in int_pos:
        left_i = i - 8
        right_i = i + 7
        if left_i >= 0 and right_i <= len(ref_seq):
            window = ref_seq[left_i:right_i]
        elif left_i < 0:
            window = 'X' * abs(left_i) + ref_seq[:right_i]
        elif right_i > len(ref_seq):
            window = ref_seq[left_i:] + (right_i - len(ref_seq)) * 'X'
        out.append(str(window))
    out = ';'.join(out)
    # concatenate aa and positions without separator
    out_psite = ';'.join([i + str(j) for i, j in zip(int_aa, int_pos)])
    psite_location.append(out_psite)
    peptide_start.append(start_pos)
    fifteenmer.append(out)
    
# create a new column in filt df with the psite location
filt_df['psite_location'] = psite_location
filt_df['peptide_start'] = peptide_start
filt_df['fifteenmer'] = fifteenmer

# filter out rows with empty psite location and report the proportion of rows removed
filt_df_2 = filt_df[filt_df['psite_location'] != ''].copy()
print('Proportion of rows removed: ', 1 - filt_df_2.shape[0] / filt_df.shape[0])

Proportion of rows removed:  0.23360932944606416


In [103]:
filt_df

,Modified sequence,Experiment,N duplicates,Sequence,Length,Missed cleavages,Proteins,Leading proteins,Gene names,Protein names,...,Curve slope error,Curve top error,Curve bottom error,EC50,pEC50,Curve effect size,Regulation,psite_location,peptide_start,fifteenmer
0,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVR,ddPTM_A549_PD325901_60min_R1,2,AAAAAAAGDSDSWDADAFSVEDPVR,25,0,O75822-2;O75822-3;O75822,O75822-2,EIF3J,Eukaryotic translation initiation factor 3 sub...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,
1,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVRK,ddPTM_A549_MK2206_60min_R1,2,AAAAAAAGDSDSWDADAFSVEDPVRK,26,1,O75822-2;O75822-3;O75822,O75822-2,EIF3J,Eukaryotic translation initiation factor 3 sub...,...,7.735097e+05,0.019667,3.390913e-02,5.639320e-08,7.248773,-0.259270,NaN,S11,2,AAAAAGDSDSWDADA
2,(ac)AAAAAAAGDS(ph)DSWDADAFSVEDPVRK,ddPTM_A549_PD325901_60min_R1,2,AAAAAAAGDSDSWDADAFSVEDPVRK,26,1,O75822-2;O75822-3;O75822,O75822-2,EIF3J,Eukaryotic translation initiation factor 3 sub...,...,4.889130e-01,0.011656,1.453447e-01,1.787879e-06,5.747662,0.440656,NaN,S11,2,AAAAAGDSDSWDADA
3,(ac)AAAAAAGPSPGSGPGDS(ph)PEGPEGEAPERR,ddPTM_A549_PD325901_60min_R1,1,AAAAAAGPSPGSGPGDSPEGPEGEAPERR,29,1,Q9UID3;E9PKE5;A0A1W2PQZ2;E9PQD5;E9PRV0;E9PKX7,Q9UID3,VPS51,Vacuolar protein sorting-associated protein 51...,...,5.742683e+05,0.113484,1.255859e-01,1.740235e-09,8.759392,-1.000544,NaN,S18,2,PGSGPGDSPEGPEGE
4,(ac)AAAADS(ph)FSGGPAGVR,ddPTM_A549_PD325901_60min_R1,1,AAAADSFSGGPAGVR,15,0,I3L2E0;A0A3B3ISX4;Q96E14,I3L2E0,RMI2,RecQ-mediated genome instability protein 2,...,9.227332e-01,0.068438,2.052981e-01,1.190380e-07,6.924314,0.690429,NaN,S7,2,XMAAAADSFSGGPAG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46786,YYS(ph)DSDDELTVEQR,ddPTM_A549_MK2206_60min_R1,1,YYSDSDDELTVEQR,14,0,Q8NFC6,Q8NFC6,BOD1L1,Biorientation of chromosomes in cell division ...,...,1.674846e+03,0.026378,2.727428e-02,1.054639e-09,8.976896,-0.148489,NaN,S482,480,YLYSKYYSDSDDELT
46787,YYS(ph)DSDDELTVEQR,ddPTM_A549_PD325901_60min_R1,1,YYSDSDDELTVEQR,14,0,Q8NFC6,Q8NFC6,BOD1L1,Biorientation of chromosomes in cell division ...,...,2.386484e+05,0.011476,1.222625e-02,1.657375e-09,8.780579,-0.112934,NaN,,,
46788,YYS(ph)IDDNQNKTHDKK,ddPTM_A549_PD325901_60min_R1,1,YYSIDDNQNKTHDKK,15,2,Q8NI08-2;Q8NI08;Q8NI08-4;Q8NI08-3;Q5TF96;Q5TF97,Q8NI08-2,NCOA7,Nuclear receptor coactivator 7,...,3.788141e+01,0.008176,1.579408e-02,1.663263e-07,6.779039,-0.148791,NaN,S89,87,TELKRYYSIDDNQNK
46789,YYSDS(ph)DDELTVEQR,ddPTM_A549_MK2206_60min_R1,1,YYSDSDDELTVEQR,14,0,Q8NFC6,Q8NFC6,BOD1L1,Biorientation of chromosomes in cell division ...,...,2.288489e+07,0.015992,3.805544e+07,1.148481e-05,4.939876,-0.362771,NaN,,,


In [91]:
out = []
if start_pos == '':
    out.append('XXXXXXXXXXXXXXX')
else:
    for i in int_pos:
        left_i = i - 7
        right_i = i + 8
        if left_i >= 0 and right_i <= len(ref_seq):
            window = ref_seq[left_i:right_i]
        elif left_i < 0:
            window = 'X' * abs(left_i) + ref_seq[:right_i]
        elif right_i > len(ref_seq):
            window = ref_seq[left_i:] + (right_i - len(ref_seq)) * 'X'
        out.append(str(window))
out = ';'.join(out)

In [92]:
out

'XXXXXXXXXXXXXXX'

In [69]:
from Bio.Seq import Seq

In [70]:
if start < 0:
    new_ref = Seq("X" * (start*-1) + str(ref_seq))

In [71]:
new_ref

Seq('XXMATNPQPQPPPPAPPPPPPQPQPQPPPPPPGPGAGPGAGGAGGAGAGAGDPQ...AKR')

In [ ]:

# create a new column in filt df with the psite location
filt_df['psite_location'] = psite_location
filt_df['peptide_start'] = peptide_start
filt_df['fifteenmer'] = fifteenmer

# filter out rows with empty psite location and report the proportion of rows removed
filt_df_2 = filt_df[filt_df['psite_location'] != ''].copy()
print('Proportion of rows removed: ', 1 - filt_df_2.shape[0] / filt_df.shape[0])

# write to file
filt_df.to_csv(output_file, sep='\t', index=False)